# OSMnx + igraph

Author: [Geoff Boeing](https://geoffboeing.com/)

First install [igraph](https://igraph.org/python/) or run Jupyter from the [Docker container](https://hub.docker.com/r/gboeing/osmnx) (which already has it installed along with OSMnx and NetworkX).

  - [Overview of OSMnx](http://geoffboeing.com/2016/11/osmnx-python-street-networks/)
  - [GitHub repo](https://github.com/gboeing/osmnx)
  - [Examples, demos, tutorials](https://github.com/gboeing/osmnx-examples)
  - [Documentation](https://osmnx.readthedocs.io/en/stable/)
  - [Journal article/citation](http://geoffboeing.com/publications/osmnx-complex-street-networks/)

In [1]:
import igraph as ig
import networkx as nx
import numpy as np
import operator
import osmnx as ox
%matplotlib inline
ox.config(log_console=True)
print(ox.__version__)
print(ig.__version__)

weight = 'length'

1.0.0
0.8.3


## Construct graphs

In [2]:
# create networkx graph
G_nx = ox.graph_from_place('Piedmont, CA, USA', network_type='drive')
osmids = list(G_nx.nodes)
G_nx = nx.relabel.convert_node_labels_to_integers(G_nx)

# give each node its original osmid as attribute since we relabeled them
osmid_values = {k:v for k, v in zip(G_nx.nodes, osmids)}
nx.set_node_attributes(G_nx, osmid_values, 'osmid')

In [3]:
%%time
# convert networkx graph to igraph
G_ig = ig.Graph(directed=True)
G_ig.add_vertices(G_nx.nodes)
G_ig.add_edges(G_nx.edges())
G_ig.vs['osmid'] = osmids
G_ig.es[weight] = list(nx.get_edge_attributes(G_nx, weight).values())

CPU times: user 745 µs, sys: 0 ns, total: 745 µs
Wall time: 747 µs


In [4]:
assert len(G_nx.nodes()) == G_ig.vcount()
assert len(G_nx.edges()) == G_ig.ecount()

## Shortest paths

In [5]:
source = list(G_nx.nodes())[0]
target = list(G_nx.nodes())[-1]

In [6]:
%%time
path1 = G_ig.get_shortest_paths(v=source, to=target, weights=weight)[0]

CPU times: user 62 µs, sys: 0 ns, total: 62 µs
Wall time: 64.4 µs


In [7]:
%%time
path2 = nx.shortest_path(G_nx, source, target, weight=weight)

CPU times: user 143 µs, sys: 0 ns, total: 143 µs
Wall time: 145 µs


In [8]:
assert path1 == path2

In [9]:
%%time
path_length1 = G_ig.shortest_paths(source=source, target=target, weights=weight)[0][0]

CPU times: user 52 µs, sys: 0 ns, total: 52 µs
Wall time: 54.6 µs


In [10]:
%%time
path_length2 = nx.shortest_path_length(G_nx, source, target, weight)

CPU times: user 149 µs, sys: 0 ns, total: 149 µs
Wall time: 151 µs


In [11]:
assert path_length1 == path_length2

## Centrality analysis

In [12]:
%%time
closeness1 = G_ig.closeness(vertices=None, mode='ALL', cutoff=None, weights=weight, normalized=True)
max_closeness1 = np.argmax(closeness1)

CPU times: user 17 ms, sys: 925 µs, total: 17.9 ms
Wall time: 17.9 ms


In [13]:
%%time
closeness2 = nx.closeness_centrality(G_nx, distance=weight, wf_improved=True)
max_closeness2 = max(closeness2.items(), key=operator.itemgetter(1))[0]

CPU times: user 251 ms, sys: 2.46 ms, total: 254 ms
Wall time: 253 ms


In [14]:
# confirm same node has max closeness in both graphs
assert G_nx.nodes[max_closeness2]['osmid'] == G_ig.vs[max_closeness1]['osmid']